# Exploring the datasets
- FUSER
- METAR
- Terminal Aerodrome Forecast (TAF)
- Convective Weather Avoidance Model (CWAM)

Save the result of the initial exploration into delta live tables for quick access

## Locate files in mounted DBFS

In [0]:
# !pip install autoviz

In [0]:
!pip install -q handyspark 

In [0]:
dbutils.library.restartPython()

In [0]:
dbutils.fs.ls("dbfs:/mnt/nasa_challenge/1-raw-unzipped-files/")

## Submission format

In [0]:
import pandas as pd
import re

In [0]:
submission_df = spark.read.csv("dbfs:/mnt/nasa_challenge/1-raw-unzipped-files/submission_format.csv", header=True, inferSchema=True)

In [0]:
submission_pd_df = submission_df.toPandas()

In [0]:
submission_pd_df.info()

In [0]:
submission_pd_df.sample(5)

Example KMEM_221129_0500_135	  99
Where KMEM is the ICAO code for the airport, 221129 is the date (YYMMDD), 0500 is the time of the prediction (HHMM) (24 hour format) and 15, 30,.. are the buckets into the future (from 1:00 to 1:15, from 1:15 to 1:30, etc.).
The order of the predictions should be the same as in the submission_format.csv file. Failing to follow this format will result in an error or lower score.

The evaluation metric is based on Root Mean Squared Error (RMSE). Specifically, it is calculated as exp(-RMSE/K), where K is a normalization factor of 10, and exp is the exponential function in order for it to be a number between 0.0 and 1.0, with 1.0 representing a perfect score (RMSE = 0).


In [0]:
# all the throughputs are set. to 99
submission_pd_df.nunique()

- ID is the time format of the prediction
- value is the throughput in the number of flights per the time period

In [0]:
def extract_info(entry):
    pattern = r"([A-Z]{4})_(\d{2})(\d{2})(\d{2})_(\d{2})(\d{2})_(\d{2,3})"
    match = re.match(pattern, entry)
    if match:
        city, year, month, day, hour, minute, buc = match.groups()
        return {
            "city": city,
            "year": "20{}".format(year),
            "month": month,
            "day": day,
            "hour": hour,
            "minute": minute,
            "buc": buc
        }
    else:
        return None

def extract_entry(entry, feature_name):
    extracted_info = extract_info(entry)    
    if extracted_info:
        return extracted_info[feature_name]
    else:
        return extracted_info
    

submission_pd_df['city'] = submission_pd_df['ID'].apply(extract_entry, feature_name="city")
submission_pd_df['year'] = submission_pd_df['ID'].apply(extract_entry, feature_name="year")
submission_pd_df['month'] = submission_pd_df['ID'].apply(extract_entry, feature_name="month")
submission_pd_df['day'] = submission_pd_df['ID'].apply(extract_entry, feature_name="day")
submission_pd_df['hour'] = submission_pd_df['ID'].apply(extract_entry, feature_name="hour")
submission_pd_df['minute'] = submission_pd_df['ID'].apply(extract_entry, feature_name="minute")
submission_pd_df['min_bucket'] = submission_pd_df['ID'].apply(extract_entry, feature_name="buc").astype(int)
submission_pd_df['prediction_datetime'] = pd.to_datetime(submission_pd_df[['year', 'month', 'day', 'hour', 'minute']])
submission_pd_df['extracted_info'] = submission_pd_df['ID'].apply(extract_info)

display(submission_pd_df)

In [0]:
# get the unique cities
cities = submission_pd_df['city'].unique()
# get the unique years
years = submission_pd_df['year'].unique()
# get start and end prediction dates in the data
first_pred_datetime = submission_pd_df['prediction_datetime'].min()
last_pred_datetime = submission_pd_df['prediction_datetime'].max()
# get the longest lookahead bucket
max_lookahead_bucket_in_min = submission_pd_df['min_bucket'].max()
min_lookahead_bucket_in_min = submission_pd_df['min_bucket'].min()
print("The cities in the dataset are: {}".format(cities))
print("The years in the dataset are: {}".format(years))
print("The predictions go from {} to {}".format(first_pred_datetime, last_pred_datetime))
print("We are looking {}-{} minutes into the future".format(min_lookahead_bucket_in_min, max_lookahead_bucket_in_min))
print("This is 15 minutes to 3 hours")

## FUSER
 The fuser data is airport specific:
 - KATL (Atlanta)
 - KCLT (Charlotte)
 - KDEN (Denver)
 - KDFW (Dallas Fort Worth)
 - KJFK (New York JFK)
 - KMEM (Memphis)
 - KORD (Chicago O'Hare)
 - KPHX (Phoenix)
 - KSEA (Seattle)

 The data also contains the test set *FUSER_test*


In [0]:
import pandas as pd

In [0]:
dbutils.fs.ls("dbfs:/mnt/nasa_challenge/1-raw-unzipped-files/FUSER_train_KATL/KATL/")

FUSER data can be further broken down into: 
- Configs Data Set (D-ATIS Data) -- configs_data_set
- Runways Data Set (Arrival Departure Detection) TARGET. --runways_data_set
- First Position Data Set -- first_position_data_set
- TBFM Data Set (Time-Based Flow Management Data) -- TBFM_data_set
- TFM Track Data Set (Traffic Flow Management Data) -- TFM_track_data_set
- ETD Data Set (Estimated Time of Departure) -- ETD_data_set
- LAMP Data Set (Local Aviation MOS Program Data) -- LAMP_data_set
- MFS Data Set (FAA SWIM Feeds) -- MFS_data_set

In [0]:
fuser_data_types = [
    'configs_data_set',
    'runways_data_set',
    'first_position_data_set',
    'TBFM_data_set',
    'TFM_track_data_set',
    'ETD_data_set',
    'LAMP_data_set',
    'MFS_data_set',
]
list_of_airport_centers = ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KORD', 
                           'KPHX', 'KSEA'] 
fuser_df = {airport_center:{} for airport_center in list_of_airport_centers}

In [0]:
# create a dictionary of dataframe for each airport and each data table in FUSER
for airport_center in list_of_airport_centers:
    for fuser_data_type in fuser_data_types:
        fuser_df[airport_center][fuser_data_type] = spark.read.csv(f"dbfs:/mnt/nasa_challenge/1-raw-unzipped-files/FUSER_train_{airport_center}/{airport_center}/*.{fuser_data_type}.csv", header=True, inferSchema=True)

In [0]:
for fuser_data_type, df in fuser_df[list_of_airport_centers[0]].items():
    num_rows = df.count()
    column_names = df.columns
    # num_distinct_dates = df.select("date_column").distinct().count()  # Replace "date_column" with the actual date column name
    print(f"Data Type: {fuser_data_type}")
    print(f"Number of Rows: {num_rows}")
    print(f"Column Names: {column_names}")
    # print(f"Number of Distinct Dates: {num_distinct_dates}")
    print("-" * 40)

In [0]:
# fields for configs_data_set
fuser_df[list_of_airport_centers[0]]['configs_data_set'].dtypes

In [0]:
for fuser_data_type, df in fuser_df[list_of_airport_centers[0]].items():
    df.describe().show()

In [0]:
for fuser_data_type, df in fuser_df[list_of_airport_centers[0]].items():
    print('Data overview')
    print(fuser_data_type)
    df.printSchema()
    print('Columns overview')
    pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])
    print('')

In [0]:
# atlanta configs_data_set
display(fuser_df[list_of_airport_centers[0]]['configs_data_set'])

Databricks data profile. Run in Databricks to view.

In [0]:
# atlanta runways_data_set
display(fuser_df[list_of_airport_centers[0]]['runways_data_set'])

Databricks data profile. Run in Databricks to view.

In [0]:
display(fuser_df[list_of_airport_centers[0]]['first_position_data_set'])

Databricks data profile. Run in Databricks to view.

In [0]:
display(fuser_df[list_of_airport_centers[0]]['TBFM_data_set'])

Databricks data profile. Run in Databricks to view.

In [0]:
display(fuser_df[list_of_airport_centers[0]]['TFM_track_data_set'])

Databricks data profile. Run in Databricks to view.

In [0]:
display(fuser_df[list_of_airport_centers[0]]['ETD_data_set'])

Databricks data profile. Run in Databricks to view.

In [0]:
display(fuser_df[list_of_airport_centers[0]]['LAMP_data_set'])

Databricks data profile. Run in Databricks to view.

In [0]:
display(fuser_df[list_of_airport_centers[0]]['MFS_data_set'])

Databricks data profile. Run in Databricks to view.

## Store data as delta tables

In [0]:
fuser_df['KCLT']['configs_data_set'].dtypes

In [0]:
fuser_df['KATL']['configs_data_set'].dtypes

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col

for airport_center in list_of_airport_centers:
    print(f"Processing {airport_center}")
    for fuser_data_type in fuser_data_types:
        df = fuser_df[airport_center][fuser_data_type]
        if fuser_data_type=='configs_data_set':
            df = df.withColumn("invalid_departure_runways", col("invalid_departure_runways").cast("double"))
            df = df.withColumn("invalid_departure_runways", col("invalid_arrival_runways").cast("double"))
        config_clt_df = config_clt_df.withColumn("invalid_arrival_runways", col("invalid_arrival_runways").cast("double"))
        table_name = f"train_{fuser_data_type}_table"
        print(f"\t Processing {fuser_data_type}")
        df.write.option("mergeSchema", "true").format("delta").mode("append").saveAsTable(table_name)